In [4]:
from bitarray import bitarray
from mmh3 import hash as mmh3_hash
import math

In [9]:
class BloomFilter:
    '''
    Bloom Filter Class
    '''
    def __init__(self, word_Nr, FPsrate:float):
        '''
        wordNr: Number of words stored in the bloom filter
        FPsrate: the FP proba
        '''
        self.FPsrate = FPsrate
        self.size = self.array_size(word_Nr, FPsrate)
        self.hashNr = self.get_hashNr(self.size, word_Nr)
        self.bit_arr = bitarray(self.size)
        # array initialized completly by 0
        self.bit_arr.setall(0)

    @classmethod
    def array_size(cls, nbr: int, FP: float):
        s = -(nbr*math.log(FP))/(math.log(2)**2)
        return int(s)
    
    # calculating the number of hash functions to be used
    @classmethod
    def get_hashNr(cls,arrsize: int,nbr_items:float):
        hash_numbers = (arrsize/nbr_items) * math.log(2)
        return int(hash_numbers)
       